# Análise de Risco: Maio/2025 - CEMADEN

Este notebook executa um pipeline completo para:
1.  Coletar e processar dados de chuva e maré.
2.  Calcular métricas de risco (`VP`, `AM`, `Nivel_Risco_Valor`).
3.  Classificar cada evento em zonas de risco (Baixo, Moderado, Moderado Alto, Alto).
4.  Gerar um relatório em texto com os pontos em cada zona.
5.  Gerar diagramas de risco interativos com Plotly para cada estação.


In [13]:
import pandas as pd
import numpy as np
from urllib.parse import quote
import os
import plotly.graph_objects as go

In [14]:
# Nomes dos arquivos no repositório do GitHub
ARQUIVO_CHUVA_GITHUB = 'Maio 2025_Cemaden_Dados Pluviométricos.csv'
ARQUIVO_MARE_GITHUB = 'mare_recife_MAIO.csv'

# Lista de estações do CEMADEN a serem analisadas
ESTACOES_DESEJADAS = ["Campina do Barreto", "Torreão", "RECIFE - APAC", "Imbiribeira"]


# Gera a lista de datas automaticamente do dia 14 ao 21 de Maio de 2025
datas_iniciais = pd.to_datetime('2025-05-14')
datas_finais = pd.to_datetime('2025-05-21')
DATAS_DESEJADAS = pd.date_range(start=datas_iniciais, end=datas_finais).strftime('%Y-%m-%d').tolist()
# --- FIM DA ALTERAÇÃO ---

# Nome da pasta onde os gráficos interativos serão salvos
PASTA_SAIDA_INTERATIVA = 'diagramas_interativos'


# Para confirmar, vamos imprimir as datas que serão analisadas:
print("Datas que serão analisadas:")
print(DATAS_DESEJADAS)

Datas que serão analisadas:
['2025-05-14', '2025-05-15', '2025-05-16', '2025-05-17', '2025-05-18', '2025-05-19', '2025-05-20', '2025-05-21']


Carregar Dados

In [15]:
def carregar_e_processar_dados(datas_desejadas, estacoes_desejadas, arquivo_chuva, arquivo_mare):
    """
    Função única para carregar todos os dados (chuva e maré), processá-los
    e retornar o DataFrame final combinado.
    """
    try:
        # --- PROCESSAMENTO DA CHUVA ---
        print("--- Etapa 1: Processando dados de CHUVA (da internet) ---")
        url_chuva = f'https://raw.githubusercontent.com/RafaellaB/Dados-Pluviom-tricos-CEMADEN/main/{quote(arquivo_chuva)}'
        df_chuva = pd.read_csv(url_chuva, sep=';', encoding='utf-8')
        df_chuva['datahora'] = pd.to_datetime(df_chuva['datahora'], errors='coerce')
        df_chuva.dropna(subset=['datahora'], inplace=True)
        df_chuva['valorMedida'] = df_chuva['valorMedida'].astype(str).str.replace(',', '.', regex=False).astype(float)

        df_filtrado = df_chuva[df_chuva['nomeEstacao'].isin(estacoes_desejadas)].copy()
        df_filtrado['data'] = df_filtrado['datahora'].dt.date.astype(str)
        df_filtrado = df_filtrado[df_filtrado['data'].isin(datas_desejadas)]

        resultados_2h = []
        df_filtrado.loc[:, 'hora_numerica'] = df_filtrado['datahora'].dt.hour
        for hora in [2, 8, 14, 20]:
            df_janela = df_filtrado[(df_filtrado['hora_numerica'] >= hora - 1) & (df_filtrado['hora_numerica'] < hora + 1)].copy()
            df_janela['janela'] = f"{hora:02d}:00:00"
            agrupado = df_janela.groupby(['nomeEstacao', 'data', 'janela'])['valorMedida'].sum().reset_index()
            resultados_2h.append(agrupado)
        df_v2horas = pd.concat(resultados_2h).rename(columns={'valorMedida': 'chuva_2h', 'janela': 'hora_ref'})

        resultados_15min = []
        for data_str in datas_desejadas:
            for hora_str in ['02:00:00', '08:00:00', '14:00:00', '20:00:00']:
                alvo = pd.to_datetime(f'{data_str} {hora_str}')
                inicio_intervalo = alvo - pd.Timedelta(minutes=15)
                df_intervalo = df_filtrado[(df_filtrado['datahora'] >= inicio_intervalo) & (df_filtrado['datahora'] < alvo)]
                soma = df_intervalo.groupby('nomeEstacao')['valorMedida'].sum().reset_index()
                soma['data'] = data_str
                soma['hora_alvo'] = hora_str
                resultados_15min.append(soma)
        df_v15min = pd.concat(resultados_15min).rename(columns={'valorMedida': 'chuva_15min', 'hora_alvo': 'hora_ref'})

        df_vp = pd.merge(df_v15min, df_v2horas, on=['data', 'hora_ref', 'nomeEstacao'], how='outer')
        df_vp[['chuva_15min', 'chuva_2h']] = df_vp[['chuva_15min', 'chuva_2h']].fillna(0)
        df_vp['VP'] = (df_vp['chuva_15min'] / 0.25) + df_vp['chuva_2h']
        print("Dados de chuva processados.")

        # --- PROCESSAMENTO DA MARÉ ---
        print("\n--- Etapa 2: Processando dados da MARÉ (da internet) ---")
        url_mare = f'https://raw.githubusercontent.com/RafaellaB/Dados-Pluviom-tricos-CEMADEN/main/{quote(arquivo_mare)}'
        dados_mare = pd.read_csv(url_mare, sep=';', encoding='latin1')
        dados_mare['data'] = pd.to_datetime(dados_mare['data'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

        dados_filtrados_mare = dados_mare[dados_mare['data'].isin(datas_desejadas)]

        resultados_am = []
        for data, grupo in dados_filtrados_mare.groupby('data'):
            alturas = grupo['altura'].tolist()
            if len(alturas) >= 2:
                primeira, segunda = alturas[0], alturas[1]
                I1 = max(primeira, segunda)
                I2 = min(primeira, segunda)
                AM = round(((I1 - I2) / 6) + I1, 2)
                resultados_am.append({'data': data, 'AM': AM})
        df_am = pd.DataFrame(resultados_am)
        print("Dados da maré processados.")

        # --- JUNÇÃO FINAL ---
        print("\n--- Etapa 3: Unindo dados de chuva e maré ---")
        df_final = pd.merge(df_vp, df_am, on='data', how='left')
        df_final.sort_values(by=['data', 'nomeEstacao', 'hora_ref'], inplace=True, ignore_index=True)
        print("DataFrames unidos com sucesso.")

        return df_final

    except Exception as e:
        print(f"\nOcorreu um erro durante o carregamento de dados: {e}")
        return None

Função Analisa Risco e Gera Relatória

In [16]:
def analisar_e_relatar_risco(df_final):
    """
    Calcula o nível de risco, classifica e imprime um relatório no console
    de acordo com as definições do TCC de Igor.
    """
    if df_final is None or df_final.empty:
        return None

    print("\n--- Etapa 4: Análise e Classificação de Risco por Ponto ---")

    df_analise = df_final.copy()
    df_analise['VP'] = df_analise['VP'].round(2)
    df_analise['AM'] = df_analise['AM'].round(2)
    df_analise['Nivel_Risco_Valor'] = (df_analise['VP'] * df_analise['AM']).fillna(0).round(2)

    bins = [-np.inf, 30, 50, 100, np.inf]
    labels = ['Baixo', 'Moderado', 'Moderado Alto', 'Alto']

    df_analise['Classificacao_Risco'] = pd.cut(df_analise['Nivel_Risco_Valor'], bins=bins, labels=labels, right=False)

    print("\n--- RELATÓRIO DE PONTOS POR ZONA DE RISCO (TCC) ---")
    for zona in ['Alto', 'Moderado Alto', 'Moderado', 'Baixo']:
        pontos_na_zona = df_analise[df_analise['Classificacao_Risco'] == zona]
        print(f"\n>> Pontos na Zona de Risco '{zona}': {len(pontos_na_zona)} ponto(s)")

        if not pontos_na_zona.empty:
            # A função display é mais elegante que print para tabelas no Jupyter/Colab
            display(pontos_na_zona[['data', 'hora_ref', 'nomeEstacao', 'Nivel_Risco_Valor']])
        else:
            print("   Nenhum ponto encontrado nesta zona.")

    return df_analise

Função - Gerar Diagramas Interativos

In [17]:
# ==============================================================================
# FUNÇÃO 3: GERAR OS DIAGRAMAS
# ==============================================================================
def gerar_diagramas_interativos_plotly(df_analisado, pasta_saida):
    """
    Pega o DataFrame analisado e gera diagramas de risco interativos com o Plotly
    """
    if df_analisado is None or df_analisado.empty:
        return

    os.makedirs(pasta_saida, exist_ok=True)
    print(f"\n--- Etapa 5: Gerando diagramas interativos e salvando na pasta '{pasta_saida}' ---")

    mapa_de_cores = {'Alto': '#D32F2F', 'Moderado Alto': '#FFA500', 'Moderado': '#FFC107', 'Baixo': '#4CAF50'}
    definicoes_risco = {
        'Baixo': 'RA < 30',
        'Moderado': '30 ≤ RA < 50',
        'Moderado Alto': '50 ≤ RA < 100',
        'Alto': 'RA ≥ 100'
    }

    for estacao, grupo in df_analisado.groupby('nomeEstacao'):
        if grupo.empty:
            continue

        fig = go.Figure()

        lim_x = max(110, grupo['VP'].max() * 1.2) if not grupo.empty else 110
        lim_y = 5

        x_grid = np.arange(0, lim_x, 1)
        y_grid = np.linspace(0, lim_y, 100)
        z_grid = np.array([x * y for y in y_grid for x in x_grid]).reshape(len(y_grid), len(x_grid))

        colorscale = [[0, "#90EE90"], [30/100, "#FFD700"], [50/100, "#FFA500"], [1.0, "#D32F2F"]]

        fig.add_trace(go.Heatmap(x=x_grid, y=y_grid, z=z_grid, colorscale=colorscale, showscale=False, zmin=0, zmax=100))

        fig.add_trace(go.Scatter(x=grupo['VP'], y=grupo['AM'], mode='lines', line=dict(color='black', width=1, dash='dash'), showlegend=False, hoverinfo='none'))

        # Plota os pontos de dados reais SEM adicioná-los à legenda
        for _, ponto in grupo.iterrows():
            cor = mapa_de_cores.get(ponto['Classificacao_Risco'], 'black')
            fig.add_trace(go.Scatter(
                x=[ponto['VP']], y=[ponto['AM']], mode='markers',
                marker=dict(color=cor, size=12, line=dict(width=1, color='black')),
                hoverinfo='text',
                hovertext=f"<b>Estação:</b> {ponto['nomeEstacao']}<br>" +
                          f"<b>Data:</b> {ponto['data']}<br>" +
                          f"<b>Hora:</b> {ponto['hora_ref']}<br>" +
                          f"<b>Risco:</b> {ponto['Classificacao_Risco']} ({ponto['Nivel_Risco_Valor']})<br>" +
                          f"<b>VP:</b> {ponto['VP']}<br>" +
                          f"<b>AM:</b> {ponto['AM']}",
                showlegend=False # Importante: não mostra os pontos de dados na legenda
            ))



        # Adiciona "pontos fantasma" (fora da área visível) apenas para criar a legenda
        for risco, definicao in definicoes_risco.items():
            fig.add_trace(go.Scatter(
                x=[None], y=[None], # Não plota nenhum ponto
                mode='markers',
                marker=dict(color=mapa_de_cores[risco], size=10, symbol='square'),
                name=f"{risco}: {definicao}" # Este texto aparecerá na legenda
            ))



        fig.update_layout(
            title=f'<b>Diagrama de Risco Interativo para a Estação: {estacao}</b>',
            xaxis_title='Precipitação (milímetros)',
            yaxis_title='Altura da Maré metros)',
            xaxis=dict(range=[0, lim_x]),
            yaxis=dict(range=[0, lim_y]),
            showlegend=True, # Liga a legenda
            legend_title_text='<b>Níveis de Risco</b>'
        )

        fig.show()

        nome_arquivo_seguro = estacao.replace(" ", "_").replace("-", "")
        save_filename = os.path.join(pasta_saida, f'diagrama_interativo_{nome_arquivo_seguro}.html')
        fig.write_html(save_filename)
        print(f"  -> Diagrama interativo para '{estacao}' salvo em: {save_filename}")

Bloco de Execução Principal

In [18]:
# 1. Carrega e processa todos os dados usando os parâmetros
dados_finais = carregar_e_processar_dados(DATAS_DESEJADAS, ESTACOES_DESEJADAS, ARQUIVO_CHUVA_GITHUB, ARQUIVO_MARE_GITHUB)

# 2. Se os dados foram carregados com sucesso, continua a análise
if dados_finais is not None:
    # Analisa o risco e imprime o relatório
    dados_analisados = analisar_e_relatar_risco(dados_finais)

    # Gera os diagramas interativos com o Plotly
    if dados_analisados is not None:
        gerar_diagramas_interativos_plotly(dados_analisados, PASTA_SAIDA_INTERATIVA)

    print("\n\n✅ Processo completo!")

--- Etapa 1: Processando dados de CHUVA (da internet) ---
Dados de chuva processados.

--- Etapa 2: Processando dados da MARÉ (da internet) ---
Dados da maré processados.

--- Etapa 3: Unindo dados de chuva e maré ---
DataFrames unidos com sucesso.

--- Etapa 4: Análise e Classificação de Risco por Ponto ---

--- RELATÓRIO DE PONTOS POR ZONA DE RISCO (TCC) ---

>> Pontos na Zona de Risco 'Alto': 0 ponto(s)
   Nenhum ponto encontrado nesta zona.

>> Pontos na Zona de Risco 'Moderado Alto': 9 ponto(s)


,data,hora_ref,nomeEstacao,Nivel_Risco_Valor
17,2025-05-15,08:00:00,Campina do Barreto,57.07
20,2025-05-15,02:00:00,Imbiribeira,54.53
25,2025-05-15,08:00:00,RECIFE - APAC,53.68
29,2025-05-15,08:00:00,Torreão,64.76
34,2025-05-16,14:00:00,Campina do Barreto,55.15
42,2025-05-16,14:00:00,RECIFE - APAC,69.52
46,2025-05-16,14:00:00,Torreão,77.31
121,2025-05-21,08:00:00,RECIFE - APAC,56.38
125,2025-05-21,08:00:00,Torreão,52.06



>> Pontos na Zona de Risco 'Moderado': 6 ponto(s)


,data,hora_ref,nomeEstacao,Nivel_Risco_Valor
3,2025-05-14,20:00:00,Campina do Barreto,35.73
11,2025-05-14,20:00:00,RECIFE - APAC,33.72
15,2025-05-14,20:00:00,Torreão,30.79
43,2025-05-16,20:00:00,RECIFE - APAC,44.74
47,2025-05-16,20:00:00,Torreão,47.17
113,2025-05-21,08:00:00,Campina do Barreto,41.36



>> Pontos na Zona de Risco 'Baixo': 113 ponto(s)


...


--- Etapa 5: Gerando diagramas interativos e salvando na pasta 'diagramas_interativos' ---


  -> Diagrama interativo para 'Campina do Barreto' salvo em: diagramas_interativos/diagrama_interativo_Campina_do_Barreto.html


  -> Diagrama interativo para 'Imbiribeira' salvo em: diagramas_interativos/diagrama_interativo_Imbiribeira.html


  -> Diagrama interativo para 'RECIFE - APAC' salvo em: diagramas_interativos/diagrama_interativo_RECIFE__APAC.html


  -> Diagrama interativo para 'Torreão' salvo em: diagramas_interativos/diagrama_interativo_Torreão.html


✅ Processo completo!


Localização das Estações

In [ ]:
!pip install folium

In [ ]:
import folium

# 1. Coordenadas aproximadas para as estações em Recife
# Nota: Estas coordenadas são aproximadas para os bairros ou locais de referência.
locais_estacoes = [
    {'nome': 'Campina do Barreto', 'lat': -8.0185, 'lon': -34.8926},
    {'nome': 'Torreão', 'lat': -8.0369, 'lon': -34.8993},
    {'nome': 'RECIFE - APAC', 'lat': -8.0543, 'lon': -34.8812}, # Coordenadas do centro do Recife como referência para a APAC
    {'nome': 'Imbiribeira', 'lat': -8.1014, 'lon': -34.9082}
]

# 2. Cria o mapa, centralizado em Recife
# A latitude e longitude inicial é o centro aproximado de Recife. Zoom inicial de 12.
mapa_recife = folium.Map(location=[-8.05, -34.90], zoom_start=12, tiles="OpenStreetMap")

# 3. Adiciona um marcador para cada estação
for estacao in locais_estacoes:
    folium.Marker(
        location=[estacao['lat'], estacao['lon']],
        # Texto que aparece ao clicar no marcador
        popup=f"<b>{estacao['nome']}</b>",
        # Texto que aparece ao passar o mouse sobre o marcador
        tooltip=estacao['nome'],
        icon=folium.Icon(color='blue', icon='cloud') # Ícone de nuvem azul
    ).add_to(mapa_recife)

# 4. Salva o mapa em um arquivo HTML
caminho_mapa = 'mapa_estacoes.html'
mapa_recife.save(caminho_mapa)
print(f"✅ Mapa interativo salvo com sucesso em: '{caminho_mapa}'")

# 5. Exibe o mapa diretamente no notebook
mapa_recife

✅ Mapa interativo salvo com sucesso em: 'mapa_estacoes.html'


atualização com as áreas de risco com os limites corretos